<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/FaultRepair2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00


In [2]:
# ─── SETUP FOR FAULT CONTROLLERS ───

import os, sys, subprocess

try:
    REPO_DIR = "/content/Cloud_Computing"
    LOGIC_DIR = os.path.join(REPO_DIR, "logic")

    # Step 1: Clone the repo if not present
    if not os.path.isdir(REPO_DIR):
        subprocess.run([
            "git", "clone", "-b", "develop",
            "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
            REPO_DIR
        ], check=True)

    # Step 2: Pull latest changes from develop
    subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

    # Step 3: Add logic folder to sys.path
    sys.path.append(LOGIC_DIR)

    # Step 4: Install dependencies
    %pip install -q importnb
    %pip install -q paho-mqtt
    %pip install -q -U gradio
    %pip install -q firebase
    %pip install requests beautifulsoup4
    %pip install -q matplotlib

    # Step 5: Import required notebooks
    from importnb import Notebook
    with Notebook():
        import Fault_controller as fc
        import repair_controller as rc

    print("✅ Fault setup completed successfully.")

except Exception as e:
    print("❌ Fault setup failed:", str(e))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.0 MB/s eta 0:00:00
omer_cohen 050-111-2222 200
shira_gold 050-222-3333 200
daniel_bar 050-333-4444 200
roni_benami 050-444-5555 200
lior_levi 050-555-6666 200
yael_elyashiv 050-666-7777 200
noa_katz 050-777-8888 200
matan_tal 050-888-9999 200
or_peled 050-999-0000 200
tom_segal 050-000-1111 200
Daniel Bar | 050-333-4444 | Robotics Engineer
Lior Levi | 050-555-6666 | Backend Developer
Matan Tal | 050-888-9999 | Mechanical Engineer
Noa Katz | 050-777-8888 | UX Designer
Omer Cohen | 050-111-2222 | Automation Engineer
Or Peled | 050-999-0000 | System Engineer
Roni Ben Ami | 050-444-5555 | QA Engineer
Shira Gold | 050-222-3333 | Electrical Engineer
Tom Segal | 050-000-1111 | Production Engineer
Yael Elyashiv | 050-666-7777 | Hardware Engineer
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 

In [6]:
import gradio as gr
from datetime import datetime

# ---------- Active faults Helpers ----------

def format_datetime(iso_str):
    dt = datetime.fromisoformat(iso_str)
    return dt.strftime("%-d/%-m, %-I:%M %p")


def get_main_fault():
    faults = fc.fb.get_active_faults()
    if not isinstance(faults, dict) or not faults:
        return None, "", "", ""

    first_key = next(iter(faults))  # This is the actual Firebase key
    fault = faults[first_key]

    title_words = fault.get("title", "").split()
    sensor1 = " ".join(title_words[:2])
    sensor2 = " ".join(title_words[2:]) if len(title_words) > 2 else ""

    return fault, sensor1, sensor2, first_key


def get_sort_key(fault, sort_mode, sensor_order):
    severity_rank = {"Low": 1, "Medium": 2, "High": 3}
    sev = severity_rank.get(fault.get("severity", ""), 0)

    if sensor_order == "Indoor → Outdoor":
        sensor_val = 0 if "Indoor" in fault.get("sensor", "") or "Indoor" in fault.get("title", "") else 1
    elif sensor_order == "Outdoor → Indoor":
        sensor_val = 1 if "Indoor" in fault.get("sensor", "") or "Indoor" in fault.get("title", "") else 0
    else:
        sensor_val = 0

    if sort_mode == "Low → High":
        return (sev, sensor_val)
    elif sort_mode == "High → Low":
        return (-sev, sensor_val)
    else:  # By Time
        return (sensor_val,) if sensor_order != "Original Order" else ()


def active_faults_table_data(sort_mode="By Time", sensor_order="Original Order", main_fault_title=None):
    faults = fc.fb.get_active_faults()
    if not isinstance(faults, dict) or not faults:
        return None, []

    fault_items = list(faults.items())

    # Exclude the current main fault
    if main_fault_title:
        fault_items = [item for item in fault_items if item[0] != main_fault_title]

    if sort_mode != "By Time" or sensor_order != "Original Order":
        fault_items = sorted(
            fault_items,
            key=lambda x: get_sort_key(x[1], sort_mode, sensor_order)
        )

    rows = []  # Will contain the final table rows (for UI display)
    keys = []  # Will store the corresponding fault keys

    for key, fault in fault_items:
        iso_ts = fault.get("timestamp", "")
        try:
            dt = datetime.fromisoformat(iso_ts)
            time_str = dt.strftime("%-d/%-m, %-I:%M%p").upper()
        except Exception:
            time_str = iso_ts[:16].replace("T", " ")

        # Append the fault key and display row to their respective lists
        keys.append(key)
        rows.append([
            time_str,
            fault.get("title", ""),
            fault.get("severity", ""),
            fault.get("status", "")
        ])

    # Return the list of fault keys and their corresponding table rows
    return keys, rows


def on_table_click(evt: gr.SelectData, keys):
    row_index = evt.index[0]
    main_key = keys[row_index]
    faults = fc.fb.get_active_faults()
    fault = faults.get(main_key, {})
    severity = fault.get("severity", "")
    severity_class = {
        "Low": "fault-low",
        "Medium": "fault-medium",
        "High": "fault-high"
    }.get(severity, "fault-low")

    return (
        main_key,
        *update_repair_box(main_key),
        *update_fault_info(main_key),
        gr.update(elem_classes=[severity_class])
    )


def update_repair_box(main_key):
    # Get all currently active faults from Firebase (or fault controller)
    faults = fc.fb.get_active_faults()
    # Find the specific fault by its key, if faults exist
    fault = faults.get(main_key) if faults else None

    # If no fault found, return updates that disable or clear the repair UI elements
    if not fault:
        return (
            gr.update(value="No Active Faults To Repair"),  # XP display text
            gr.update(value=""),                            # Sensor name text cleared
            gr.update(visible=False),                       # Hide start button
            gr.update(choices=[], value=[], interactive=False),  # Clear and disable checklist
            gr.update(visible=False),                       # Hide finish button
        )

    # Map fault severity to XP reward values
    severity_to_xp = {"Low": 50, "Medium": 100, "High": 200}
    # Get XP based on severity, default to 50 if unknown severity
    xp = severity_to_xp.get(fault.get("severity", "Low"), 50)

    title = fault.get("title", "")
    title_words = title.split()
    # Extract first two words from title for sensor snippet display
    sensor_snippet = " ".join(title_words[:2]) if len(title_words) >= 2 else title

    # Return UI updates to show XP, sensor info, enable start button, checklist with actions,
    # and keep finish button hidden initially
    return (
        gr.update(value=f"Earn {xp} XP by fixing"),                             # XP reward text
        gr.update(value=f"{sensor_snippet} Sensor Whithin 10 Minutes"),        # Sensor name text
        gr.update(visible=True),                                                # Show start button
        gr.update(choices=fault["actions"], value=[], interactive=False),       # Checklist with fault actions, no selection yet
        gr.update(visible=False),                                               # Hide finish button initially
    )


def update_fault_info(main_key):
    # Get current active faults
    faults = fc.fb.get_active_faults()
    # Get specific fault by key
    fault = faults.get(main_key) if faults else None

    # If fault not found, return placeholders indicating no data
    if not fault:
        return (
            gr.update(value="--"),       # fault time placeholder
            gr.update(value="--"),       # fault location placeholder
            gr.update(value="No actions available.")  # fault actions placeholder text
        )

    # Parse ISO timestamp and format it to readable string (e.g., 5/22, 3:45PM)
    iso_ts = fault.get("timestamp", "")
    try:
        dt = datetime.fromisoformat(iso_ts)
        time_str = dt.strftime("%-d/%-m, %-I:%M%p").upper()
    except Exception:
        # Fallback formatting if parsing fails
        time_str = iso_ts[:16].replace("T", " ")

    title = fault.get("title", "")
    actions = fault.get("actions", [])

    # Format list of actions as bullet points for display
    action_lines = "\n".join(f"- {a}" for a in actions)

    # Return UI updates for fault info display elements
    return (
        gr.update(value=f"**{time_str}**"),   # Formatted fault time
        gr.update(value=f"### {title}"),      # Fault title
        gr.update(value=action_lines)          # List of fault actions
    )


# ----------- Resolved faults Helper -----------
def resolved_faults_table_data():
    try:
        faults = fc.fb.get_resolved_faults()
        if not isinstance(faults, dict) or not faults:
            return []

        rows = []
        for _, fault in faults.items():
            try:
                time_str = format_datetime(fault.get("timestamp", ""))
            except:
                time_str = fault.get("timestamp", "")[:16].replace("T", " ")

            rows.append([
                time_str,
                fault.get("sensor", ""),
                fault.get("severity", ""),
                fault.get("repaired_by", ""),
                fault.get("status", "Resolved")
            ])
        return rows
    except Exception as e:
        print("❌ resolved_faults_table_data() error:", e)
        return []


# ---------- Extracted ----------
presented_fault, sensor1, sensor2, presented_key = get_main_fault()
time_str = format_datetime(presented_fault["timestamp"])
xp_value = {"Low": 50, "Medium": 100, "High": 200}.get(presented_fault["severity"], 50)
severity_class = {
    "Low": "fault-low",
    "Medium": "fault-medium",
    "High": "fault-high"
}.get(presented_fault.get("severity", "Low"), "fault-low")

# ---------- Styles ----------
css = f"""
.fault-low {{
    background-color: #fff176;
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
.fault-medium {{
    background-color: #ffb74d;
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
.fault-high {{
    background-color: #ef5350;
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#main-fault * {{
    color: black !important;
}}

#repair-box {{
    background-color: white;
    color: black;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#repair-box * {{
    color: black !important;
}}
.repair-checkbox {{
  display: flex !important;
  flex-direction: row !important;
  gap: 20px;
  flex-wrap: wrap;
}}
"""

# ---------- App ----------
with gr.Blocks(css=css) as demo:

    # -------- First Row ----------
    with gr.Row():
      main_fault_column = gr.Column(elem_id="main-fault", elem_classes=[severity_class])
      with main_fault_column:
             gr.Markdown("## FAULT DETECTED")
             fault_time_text = gr.Markdown("")
             fault_sensor_name = gr.Markdown("")
             gr.Markdown("#### Suggested Actions:")
             fault_actions_box = gr.Markdown("")

      with gr.Column(elem_id="repair-box", scale=1):
            gr.Markdown("## REPAIR CHALLENGE")
            repair_xp_text = gr.Markdown("")
            repair_sensor_text = gr.Markdown("")
            gr.Markdown("#### Repair Steps")

            repair_checklist = gr.CheckboxGroup(
                choices=[],
                value=[],
                interactive=False,
                label="",
                elem_classes=["repair-checklist"]
            )

            start_btn = gr.Button("Start Repair", visible=True)
            finish_btn = gr.Button("Finish Repair", visible=False, interactive=False)
            status_output = gr.Textbox(label="System Message", interactive=False)

    # -------- Second Row ----------
    with gr.Row():
        with gr.Column():
            with gr.Row():
                gr.Markdown("### Active Faults")

                sensor_filter = gr.Dropdown(
                    choices=["Original Order", "Indoor → Outdoor", "Outdoor → Indoor"],
                    value="Original Order",  # default
                    label="",
                    interactive=True,
                    scale=1,
                    elem_classes=["small-dropdown"]
                )
                sort_choice = gr.Dropdown(
                    choices=["By Time", "Low → High", "High → Low"],
                    value="By Time",      # default
                    label="",
                    interactive=True,
                    scale=1,
                    elem_classes=["small-dropdown"]
                )


            table_headers = gr.Dataframe(
                headers=["Time", "Sensor", "Severity", "Status"],
                value=[],
                row_count=0,
                col_count=(4, "fixed"),
                interactive=True,
                label=""
        )
    # -------- Third Row ----------
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Resolved Faults")
            resolved_table = gr.Dataframe(
                headers=["Time Detected", "Sensor", "Severity", "Repaired by", "Status"],
                value=[],                     # Start empty
                row_count=0,
                col_count=(5, "fixed"),
                interactive=False,
                label=""
            )

    # ---------- Logic ----------
    def start_real_repair(main_key):
        faults = fc.fb.get_active_faults()                                      # Get the active fault dictionary from Firebase
        fault = faults.get(main_key)                                            # Get the specific fault by key

        if not fault or not isinstance(fault, dict):
            return (
                gr.update(choices=[], value=[], interactive=False),             # checklist
                gr.update(visible=True),                                        # start_btn
                gr.update(visible=False)                                        # finish_btn
            )

        # Make sure actions are a list
        actions = fault.get("actions")
        # Start the repair (write to Firebase)
        msg = rc.start_repair(main_key)

        return (
            gr.update(choices=actions, value=[], interactive=True),             # checklist
            gr.update(visible=False),                                           # start_btn
            gr.update(visible=True),                                            # finish_btn
            msg                                                                 # status_output
        )

    def handle_checkbox_update(selected, main_key):
        faults = fc.fb.get_active_faults()
        fault = faults[main_key]
        expected = fault.get("actions", [])
        return gr.update(interactive=(selected == expected))


    def complete_real_repair(main_key):
        msg = rc.complete_repair(main_key)  # Again, use the Firebase key
        return (
            gr.update(visible=False),
            gr.update(value="Start Repair", visible=True, interactive=True),
            msg
        )


    def load_active_faults():
        rows = active_faults_table_data()
        return gr.update(value=rows)


    # ---------- Bindings ----------
    presented_key_state = gr.State(presented_key)
    fault_keys_state = gr.State([])

    start_btn.click(
        fn=start_real_repair,
        inputs=[presented_key_state],
        outputs=[repair_checklist, start_btn, finish_btn, status_output]
    )

    finish_btn.click(
        fn=complete_real_repair,
        inputs=[presented_key_state],
        outputs=[finish_btn, start_btn, status_output]
    )

    repair_checklist.change(
        fn=handle_checkbox_update,
        inputs=[repair_checklist, presented_key_state],
        outputs=finish_btn
    )

    sort_choice.change(
        fn=lambda sort, sensor, key: active_faults_table_data(sort, sensor, key)[1],
        inputs=[sort_choice, sensor_filter, presented_key_state],
        outputs=[table_headers]
    )

    sensor_filter.change(
        fn=lambda sort, sensor, key: active_faults_table_data(sort, sensor, key)[1],
        inputs=[sort_choice, sensor_filter, presented_key_state],
        outputs=[table_headers]
    )

    # Register a selection event on the fault table
    table_headers.select(
    # Function to call when a row is selected
    fn=on_table_click,
    inputs=[fault_keys_state],
    outputs=[
        presented_key_state,   # Update the state of the currently presented fault key
        repair_xp_text,        # Update the XP (score) text area for the selected fault
        repair_sensor_text,    # Update the sensor name or location text for the selected fault
        start_btn,
        repair_checklist,      # Load and show the checklist steps (if any) for the fault
        finish_btn,
        fault_time_text,
        fault_sensor_name,
        fault_actions_box,
        main_fault_column,
        ]
    )


    demo.load(
    fn=lambda key: (
        *update_repair_box(key),
        active_faults_table_data("By Time", "Original Order", key)[1],  # rows
        active_faults_table_data("By Time", "Original Order", key)[0],  # keys
        resolved_faults_table_data(),
        *update_fault_info(key)
    ),
    inputs=[presented_key_state],
    outputs=[
        repair_xp_text,
        repair_sensor_text,
        start_btn,
        repair_checklist,
        finish_btn,
        table_headers,
        fault_keys_state,
        resolved_table,
        fault_time_text,         # UI elements at top
        fault_sensor_name,
        fault_actions_box
    ]
)




demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa99dc74d15640bdef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
